In [37]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re

In [38]:
def rmse_loss(input, target, mask, reduction="mean"):
        out = (input[~mask]-target[~mask])**2
        return out.mean().sqrt() if reduction == "mean" else out 

def mae_loss(input, target, mask, reduction="mean"):
        out = (input[~mask]-target[~mask])
        return out.mean().abs() if reduction == "mean" else out 

In [39]:

p = '/data1/jessica/data/toyota/ckpts_final/'

experiments = os.listdir(p)
res = {}
res_ckpt = {}
for elem in experiments:
    path = p + elem + "/lightning_logs/" 
    vs = os.listdir(path)
    filt = []
    for elem1 in vs: 
        if 'version' in elem1:
            filt.append(elem1)
    versions =[int(elem.split("_")[-1])for elem in filt]
    versions = sorted(versions)
    version = f"version_{versions[-1]}"
    print(versions, version)
    if elem == 'ckpts_final_comma_distance_clip' or elem == 'ckpts_final_comma_multitask_vit': version = "version_0"
    

    checkpoint_path = path + version + "/checkpoints/"
    if "checkpoints" not in os.listdir(path + version): continue
    files = os.listdir(checkpoint_path)

    task = []
    ckpt = []
    files = sorted(files)
    for filename in files: 
        if filename.endswith(".csv"):
            df = pd.read_csv(checkpoint_path + filename)
            df.columns = ['preds', 'targets']
            m = (df['targets'] == 0).astype(bool)  if "angle" not in elem  else (df['targets'] == np.inf).astype(bool)
            
            loss3 = rmse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m))
            task.append(round(loss3.item(), 2))
        if filename.endswith(".ckpt"):
            ckpt.append(checkpoint_path + '/' + filename)
    res[elem] = task
    res_ckpt[elem] = ckpt


[0] version_0
[0] version_0
[6, 7, 8] version_8
[3, 4, 5] version_5
[4, 5, 6] version_6
[0] version_0
[0] version_0
[7, 8, 9] version_9
[5, 6, 7] version_7
[0, 1, 2, 3, 4] version_4
[5, 6, 7] version_7
[6, 7, 8, 9, 10] version_10
[0] version_0
[5, 6, 7, 8, 9] version_9
[2, 3, 4] version_4
[0] version_0
[1] version_1
[0] version_0
[6, 7, 8, 9, 10, 11, 12, 13, 14, 15] version_15
[0, 2] version_2
[10] version_10
[2, 3, 4] version_4
[6] version_6
[2] version_2


In [40]:
rows = []
for elem in res.keys():
    splitted = elem.split("_")
    data = splitted[2]
    task = splitted[3]
    backbone = splitted[4]
    rows.append({"Task": task, "Backbone": backbone, "Dataset": data, "t-RMSE": round(res[elem][0], 2) if len(res[elem]) == 1 else res[elem]})



In [41]:
df = pd.DataFrame(rows)
df

,Task,Backbone,Dataset,t-RMSE
0,distance,resnet,comma,11.07
1,multitask,none,comma,"[4.76, 6.34]"
2,angle,vit,nuscenes,29.2
3,multitask,none,nuscenes,"[17.75, 13.24]"
4,multitask,resnet,nuscenes,"[39.25, 42.25]"
5,multitask,resnet,comma,"[2.93, 9.38]"
6,angle,resnet,comma,3.28
7,multitask,vit,nuscenes,"[26.14, 27.43]"
8,angle,resnet,nuscenes,31.99
9,distance,vit,nuscenes,[]


In [42]:
# The list of dictionaries
data = rows

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame
df_pivot = pd.pivot_table(df, values='t-RMSE', index=['Dataset', 'Backbone'], columns='Task', aggfunc=lambda x: x)

# Rename the columns
df_pivot.columns = [f'{col}-MAE' for col in df_pivot.columns]
print(df_pivot)

# Reset the index
df_pivot = df_pivot.reset_index()
df_pivot['Feat. Size'] = [512, 643, 512, 768, 512, 643, 512, 768]
df_pivot = df_pivot[["Dataset", "Backbone", 'Feat. Size', 'angle-MAE', 'distance-MAE', 'multitask-MAE']].round(2)
# Display the pivoted DataFrame
print(df_pivot.to_latex(index=False))


                  angle-MAE distance-MAE   multitask-MAE
Dataset  Backbone                                       
comma    clip          2.12        12.42   [2.71, 11.83]
         none          5.26         8.49    [4.76, 6.34]
         resnet        3.28        11.07    [2.93, 9.38]
         vit           2.99        12.69    [2.99, 9.38]
nuscenes clip          17.0         10.7  [18.94, 12.22]
         none            []         8.45  [17.75, 13.24]
         resnet       31.99        28.68  [39.25, 42.25]
         vit           29.2           []  [26.14, 27.43]
\begin{tabular}{llrlll}
\toprule
 Dataset & Backbone &  Feat. Size & angle-MAE & distance-MAE &  multitask-MAE \\
\midrule
   comma &     clip &         512 &      2.12 &        12.42 &  [2.71, 11.83] \\
   comma &     none &         643 &      5.26 &         8.49 &   [4.76, 6.34] \\
   comma &   resnet &         512 &      3.28 &        11.07 &   [2.93, 9.38] \\
   comma &      vit &         768 &      2.99 &        12.69 &  

/tmp/ipykernel_1691964/1351012582.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_pivot.to_latex(index=False))


In [43]:
df_pivot

,Dataset,Backbone,Feat. Size,angle-MAE,distance-MAE,multitask-MAE
0,comma,clip,512,2.12,12.42,"[2.71, 11.83]"
1,comma,none,643,5.26,8.49,"[4.76, 6.34]"
2,comma,resnet,512,3.28,11.07,"[2.93, 9.38]"
3,comma,vit,768,2.99,12.69,"[2.99, 9.38]"
4,nuscenes,clip,512,17.0,10.7,"[18.94, 12.22]"
5,nuscenes,none,643,[],8.45,"[17.75, 13.24]"
6,nuscenes,resnet,512,31.99,28.68,"[39.25, 42.25]"
7,nuscenes,vit,768,29.2,[],"[26.14, 27.43]"
